In [21]:
import gradio as gr
import ast
from pynput import keyboard

In [22]:
def update_macros(action, keys, macro_to_delete, current_macros):
    # Check if current_macros is a string representation of a dictionary, and convert it if so
    if isinstance(current_macros, str) and current_macros.strip():
        # Safely convert the string representation back to a dictionary
        current_macros = ast.literal_eval(current_macros)
    else:
        # Initialize as an empty dictionary if not a valid string representation
        current_macros = {}
    
    # Check if a user entered a macro to delete
    if macro_to_delete and macro_to_delete in current_macros:
        del current_macros[macro_to_delete]  
    elif action and keys:  # Ensure non-empty input
        # Update or add the macro definition
        newlist = key_list.copy()
        current_macros[action] = {'keys': newlist}

    # Format the macros for display in a readable format
    macros_text = ""
    for act, vals in current_macros.items():
        macros_text += f"[{act}]  ->  {vals['keys']}\n"

    clear_list()

    # Return both the formatted string for display and the updated macros dictionary, Clear the display for action_input, keys_input, and macro_to_delte
    return macros_text, str(current_macros), "", ""

In [23]:
key_list = []

def read_keys():
    def on_press(key):
        try:
            print('alphanumeric key {0} pressed'.format(key.char))
            if key == keyboard.Key.esc:
                return False
        except AttributeError:
            print('special key {0} pressed'.format(key))
        
    def on_release(key):
        # try:
            print('{0} released'.format(key))
            key_list.append(key.char)
        # except AttributeError:
        #     key_list.append(key)
            print(key_list)

    with keyboard.Listener(
        on_press=on_press,
        on_release=on_release) as listener:
        listener.join()

    return key_list

def clear_list():
    key_list.clear()


In [24]:
with gr.Blocks() as settings_page:
    gr.Markdown("# Macro Setting")
    gr.Markdown("""
    Configure your audio command macros here. Enter a speech command, specify the keys it should trigger, and set the timing.
    """)
    
    with gr.Row():
        # Create a textbox for the speech command
        action_input = gr.Textbox(label="Speech Command") 
        # Create a textbox for the key
        keys_input = gr.Button("Keys to Simulate")

    # Gets the macro the user wants to delete
    macro_to_delete = gr.Textbox(label="Type the Speech Command to Delete")
    # Creates a button to update or add the new settings
    update_btn = gr.Button("Update/Add Macro") 
    
    # Display for the current macros
    current_macros_display = gr.Textbox(label="Current Macros", value="", lines=10, interactive=False) 
    macros_state = gr.State('{}')
    # Runs when the button is clicked updating the current macros display
    keys_input.click(read_keys)
    update_btn.click(update_macros, 
                     inputs=[action_input, keys_input,macro_to_delete, macros_state], 
                     outputs=[current_macros_display, macros_state, action_input, macro_to_delete])


In [25]:
settings_page.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


alphanumeric key r pressed
'r' released
['r']


Unhandled exception in listener callback
Traceback (most recent call last):
  File "c:\Users\jakin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pynput\_util\__init__.py", line 228, in inner
    return f(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jakin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pynput\keyboard\_win32.py", line 290, in _process
    self.on_release(key)
  File "c:\Users\jakin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pynput\_util\__init__.py", line 144, in inner
    if f(*args) is False:
       ^^^^^^^^
  File "C:\Users\jakin\AppData\Local\Temp\ipykernel_17848\3848912610.py", line 15, in on_release
    key_list.append(key.char)
                    ^^^^^^^^
AttributeError: 'Key' object has no attribute 'char'


special key Key.esc pressed
Key.esc released


Traceback (most recent call last):
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\route_utils.py", line 233, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 1608, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 1176, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\anyio\to_thread.py", line 56, in run_sync
    r

...

Traceback (most recent call last):
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\route_utils.py", line 233, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 1608, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 1176, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\anyio\to_thread.py", line 56, in run_sync
    r

alphanumeric key w pressed
'w' released
['w']
alphanumeric key a pressed
'a' released
['w', 'a']
alphanumeric key s pressed
's' released
['w', 'a', 's']


Unhandled exception in listener callback
Traceback (most recent call last):
  File "c:\Users\jakin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pynput\_util\__init__.py", line 228, in inner
    return f(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jakin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pynput\keyboard\_win32.py", line 290, in _process
    self.on_release(key)
  File "c:\Users\jakin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pynput\_util\__init__.py", line 144, in inner
    if f(*args) is False:
       ^^^^^^^^
  File "C:\Users\jakin\AppData\Local\Temp\ipykernel_17848\3848912610.py", line 15, in on_release
    key_list.append(key.char)
                    ^^^^^^^^
AttributeError: 'Key' object has no attribute 'char'


special key Key.esc pressed
Key.esc released


Traceback (most recent call last):
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\route_utils.py", line 233, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 1608, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py", line 1176, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jakin\AppData\Roaming\Python\Python312\site-packages\anyio\to_thread.py", line 56, in run_sync
    r